# 网络结构的可视化
## 准备网络和数据

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
from torch.optim import SGD
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
train_data = torchvision.datasets.MNIST(
    root="./data/MNIST", ## 数据的路径
    train=True,     ## 只使用训练数据集
    ## 将数据转化为torch使用的张量，取值范围是[0,1]
    transform=torchvision.transforms.ToTensor(),
    download=False  ## 因为数据已经过，所以这里不再下载
)


In [3]:
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=128,
    shuffle=True,
    num_workers=2,
)

In [4]:
for step, (b_x,b_y) in enumerate(train_loader):
    if step > 0:
        break

print(b_x.shape)
print(b_y.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [5]:
test_data = torchvision.datasets.MNIST(
    root = "./data/MNIST",
    train=False,
    download=False
)

In [6]:
test_data_x = test_data.data.type(torch.FloatTensor) / 255.0
test_data_x = torch.unsqueeze(test_data_x, dim=1)
test_data_y = test_data.targets
print("test_data_x.shape:",test_data_x.shape)
print("test_data_y.shape:",test_data_y.shape)

test_data_x.shape: torch.Size([10000, 1, 28, 28])
test_data_y.shape: torch.Size([10000])


In [7]:
## 搭建一个卷积神经网络
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super(ConvNet, self).__init__()
        ## 定义第一个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.AvgPool2d(
                kernel_size=2,
                stride=2,
            ),
        )
        ## 定义第二个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,3,1,1),
            nn.ReLU(),              ## 激活函数
            nn.MaxPool2d(2,2)       ## 最大池化层
        )

        ## 定义全连接层
        self.fc = nn.Sequential(
            nn.Linear(
                in_features=32*7*7,
                out_features=128,
            ),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU()
        )

        self.out = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = self.out(x)
        return output

In [8]:
## 输出网络结构
MyConvnet = ConvNet()
print(MyConvnet)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1568, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [10]:
import hiddenlayer as hl
## 可视化神经网络
hl_graph = hl.build_graph(MyConvnet, torch.zeros([1,1,28,28]))
# hl_graph.theme = hl.graph.THEMES["blue"].copy()
## 可视化的网络保存为图片
# hl_graph.save("./data/lesson4_hiddenlay.png", format="png")

TypeError: _jit_pass_onnx_unpack_quantized_weights(): incompatible function arguments. The following argument types are supported:
    1. (arg0: torch::jit::Graph, arg1: Dict[str, IValue], arg2: bool) -> Dict[str, IValue]

Invoked with: graph(%input.1 : Float(1, 1, 28, 28, strides=[784, 784, 28, 1], requires_grad=0, device=cpu),
      %1 : Float(16, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cpu),
      %2 : Float(16, strides=[1], requires_grad=1, device=cpu),
      %3 : Float(32, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=1, device=cpu),
      %4 : Float(32, strides=[1], requires_grad=1, device=cpu),
      %5 : Float(128, 1568, strides=[1568, 1], requires_grad=1, device=cpu),
      %6 : Float(128, strides=[1], requires_grad=1, device=cpu),
      %7 : Float(64, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %8 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %9 : Float(10, 64, strides=[64, 1], requires_grad=1, device=cpu),
      %10 : Float(10, 

In [15]:
from torchviz import make_dot
x = torch.randn(1,1,28,28).requires_grad_(True)
y = MyConvnet(x)
MyConvnetvis = make_dot(y, params=dict(list(MyConvnet.named_parameters()) + [('x',x)]))
MyConvnetvis.format = "png"
MyConvnetvis.directory = "./"
MyConvnetvis.view()

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH